In [1]:
import sys
sys.path.append("/Users/axl/Mispricing Model_Leviathan/leviathan-model")

import pandas as pd
import matplotlib.pyplot as plt

from src.core.pipeline import run_pipeline
from src.features.dti import attach_affordability_features


ImportError: cannot import name 'attach_affordability_features' from 'src.features.dti' (/Users/axl/Mispricing Model_Leviathan/leviathan-model/src/features/dti.py)

In [2]:
import inspect
import src.features.dti as dti

print(dir(dti))


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [3]:
import src.features.dti as dti
import inspect

print(inspect.getsource(dti))


OSError: could not get source code

In [4]:
import src.features.dti as dti
dti.__file__


'/Users/axl/Mispricing Model_Leviathan/leviathan-model/src/features/dti.py'

In [6]:
import inspect
import src.features.dti as dti
print(inspect.getsource(dti))


import pandas as pd

def compute_price_to_income(df: pd.DataFrame) -> pd.Series:
    required = {"price", "income"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"missing required columns: {missing}")
    return df["price"] / df["income"]


def compute_mortgage_burden(
    df: pd.DataFrame,
    *,
    term_years: int = 25,
    loan_to_value: float = 0.8,
) -> pd.Series:
    required = {"price", "income", "mortgage_rate"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"missing required columns: {missing}")

    price = df["price"].astype(float)
    income = df["income"].astype(float)
    r_annual = df["mortgage_rate"].astype(float)

    r_month = r_annual / 12.0
    n_months = term_years * 12
    loan = price * loan_to_value

    def monthly_payment(P, r, n):
        eps = 1e-8
        if abs(r) < eps:
            return P / n
        factor = (1 + r) ** n
        return P * r * factor / (factor - 1)

    payment =